In [1]:
# name cleanup...
# lower case
# turn ampersand to "and"
# shorten anything "incorporated" should be inc
# "limited liability company" to llc?
# other company specific weirdnesses?
# then, remove all other punct

# address parsing
# lower case
# how to parse and normalize? based on punct, state abbreve? or just keep whole string and do general cleanup?

# fuzzy wuzzy package for matching strings

# CompanyMap lazy learning object with data for seen companies
# CompanyMap.classify takes in any of name, DBA, address info and assigns an ID with confidence score.
# update for editing bad learnings


In [1]:
import sys
import os
project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

from src.mapping import CompanyMap, fuzzy_join
import pandas as pd


In [2]:
reg = pd.read_csv('../Contractor_Registry_certificate_20250215.csv')
reg

,Certificate Number,Business Name,DBA Name,Business Type,Business is MWBE Owned,Business is Publicly Traded,Business Officers,Address,Address 2,City,...,Debarment Start Date,Debarment End Date,Business has final determination for violation of Labor or Tax Law,Business has final determination safety standard violations,Business is associated with an apprenticeship program,Business is sponsor of a program,Business is signatory to a group program,Business has Workers Compensation Insurance,Business is exempt from Workers Compensation Insurance,Georeference
0,24-639SQ-CR,10100 Inc.,"10100, Inc.",Corporation,No,No,NaN,955 West River Road,NaN,Grand Island,...,NaN,NaN,No,No,No,No,No,Yes,No,POINT (-78.99049 42.97805)
1,24-63ZE1-CR,"1068 Curry Road, Inc.",city glass company,Corporation,No,No,NaN,1068 curry rd,NaN,"schenectady, ny 12306",...,NaN,NaN,No,No,No,No,No,Yes,No,POINT (-73.969 42.78614)
2,25-64TYL-CR,11400 LLC,CLARK FOOD SERVICE EQUIPMENT,LLC,No,No,NaN,2551 HORSESHOE RD,NaN,LANCASTER,...,NaN,NaN,No,No,No,No,No,Yes,No,POINT (-76.206 40.07246)
3,25-6414J-CR,"1349 Main, Gorenflo's Buffalo Wholesale Lock C...",Gorenflo's Buffalo Wholesale Lock,Corporation,No,No,NaN,1349 Main Street,NaN,Buffalo,...,NaN,NaN,No,No,No,No,No,Yes,No,POINT (-78.86594 42.91033)
4,25-64V75-CR,1895 ELECTRIC LLC,NaN,LLC,No,No,NaN,60 SCHOL ST#114,NaN,ORCHARD PARK,...,NaN,NaN,No,No,No,No,No,Yes,No,POINT (-78.74632 42.76559)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7012,24-64EIY-CR,Zoria Housing LLC,Zoria Housing LLC,LLC,Yes,No,NaN,87-28 130th street,NaN,JAMAICA,...,NaN,NaN,No,No,Yes,No,Yes,Yes,No,POINT (-73.82193 40.70079)
7013,24-64FPI-CR,"Zorn Industries, Inc.",NaN,Corporation,No,No,NaN,11 SETTLERS WAY,NaN,SETAUKET,...,NaN,NaN,No,No,No,No,No,Yes,No,POINT (-73.12543 40.93814)
7014,24-64RJL-CR,"Zuech's Environmental Services, Inc.",NaN,Corporation,No,No,NaN,PO Box 108,NaN,Franklinville,...,NaN,NaN,No,No,No,No,No,Yes,No,NaN
7015,24-6358H-CR,Zuke's Excavating LLC,NaN,LLC,No,No,NaN,65 Sanitaria Springs Road,NaN,Binghamton,...,NaN,NaN,No,No,No,No,No,Yes,No,POINT (-75.74882 42.14368)


In [3]:
reg = reg.rename(columns={'Business Name':'NAME','DBA Name':'DBA'})
reg['ADDRESS'] = reg['Address'].fillna("") + " " + reg['Address 2'].fillna("") + " " + reg['City'].fillna("") + " " + reg['State'].fillna("") + " " + reg['Zip Code'].fillna("")

In [4]:
debar = pd.read_csv('../data/processed/NYSDOL_debarment_02_19_2025.csv')
debar

,ID,AGENCY,FISCAL_OFFICER,FEIN,EMPLOYER_NAME,EMPLOYER_DBA,ADDRESS,DEBAR_START,DEBAR_END
0,1.0,DOL,DOL,*****5754,"0369 CONTRACTORS, LLC",NaN,515 WEST AVE UNIT PH 13NORWALK CT 06850,05/12/2021,05/12/2026
1,2.0,DOL,DOL,*****5784,"A.J.M. TRUCKING, INC.",NaN,PO BOX 2064 MONROE NY 10950,02/12/2024,02/12/2029
2,3.0,DOL,DOL,NaN,AKHLAQ OULAKH,NaN,4307 28TH AVE ASTORIA NY 11103,10/11/2024,10/11/2029
3,4.0,DOL,NYC,NaN,"ALL COUNTY SEWER & DRAIN, INC.",NaN,7 GREENFIELD DR WARWICK NY 10990,03/25/2022,03/25/2027
4,5.0,DOL,DOL,*****8387,"AMERICAN PAVING & MASONRY, CORP.",NaN,8 FOREST AVE GLEN COVE NY 11542,05/24/2024,05/24/2029
...,...,...,...,...,...,...,...,...,...
202,204.0,DOL,DOL,NaN,PAULINE CHAHALES,NaN,935 S LAKE BLVD MAHOPAC NY 10541,05/17/2021,05/17/2026
203,205.0,DOL,DOL,*****9060,"PEC GROUP OF N.Y., INC.",NaN,935 S LAKE BLVD SUITE 7MAHOPAC NY 10541,05/17/2021,05/17/2026
204,206.0,DOL,DOL,*****9060,"PEC GROUP OF N.Y., INC.",NaN,935 S LAKE BLVD SUITE 7MAHOPAC NY 10541,03/02/2021,03/02/2026
205,207.0,DOL,DOL,NaN,RUSSELL NEEDHAM,NaN,532 NEPTUNE AVENUE BROOKLYN NY 11224,01/12/2022,01/12/2027


In [5]:
debar = debar.rename(columns={'EMPLOYER_NAME':'NAME','EMPLOYER_DBA':'DBA'})

In [6]:
df = fuzzy_join(debar, reg, ['NAME','DBA'], addr_col='ADDRESS', how='inner')
df

,ID,AGENCY,FISCAL_OFFICER,FEIN,NAME_l,DBA_l,ADDRESS_l,DEBAR_START,DEBAR_END,variable,...,Business is associated with an apprenticeship program,Business is sponsor of a program,Business is signatory to a group program,Business has Workers Compensation Insurance,Business is exempt from Workers Compensation Insurance,Georeference,ADDRESS_r,norm_NAME,norm_DBA,norm_ADDRESS
value,,,,,,,,,,,,,,,,,,,,,
1921.0,35.0,DOL,DOL,*****7619,DANCO CONSTRUCTION UNLIMITED INC.,NaN,485 RAFT AVENUE HOLBROOK NY 11741,10/19/2021,10/19/2026,0,...,No,No,No,Yes,No,POINT (-73.0722 40.77477),485 RAFT AVENUE HOLBROOK NY 11741,d kalogeras construction inc,,485 raft avenue holbrook ny 11741


In [7]:
c = CompanyMap(reg)
c.get_match_df(['DANCO CONSTRUCTION UNLIMITED INC.'], '485 RAFT AVENUE HOLBROOK NY 11741')

,Certificate Number,NAME,DBA,Business Type,Business is MWBE Owned,Business is Publicly Traded,Business Officers,Address,Address 2,City,...,Business is associated with an apprenticeship program,Business is sponsor of a program,Business is signatory to a group program,Business has Workers Compensation Insurance,Business is exempt from Workers Compensation Insurance,Georeference,ADDRESS,norm_NAME,norm_DBA,norm_ADDRESS
1921,25-64O64-CR,D KALOGERAS CONSTRUCTION INC,NaN,Corporation,No,No,NaN,485 RAFT AVENUE,NaN,HOLBROOK,...,No,No,No,Yes,No,POINT (-73.0722 40.77477),485 RAFT AVENUE HOLBROOK NY 11741,d kalogeras construction inc,,485 raft avenue holbrook ny 11741


In [8]:
sig = pd.read_csv('../data/raw/apprentice_signatories.csv')

In [9]:
sig = sig.rename(columns={'Signatory Name':'NAME'})
sig['ADDRESS'] = sig['Signatory Address'].fillna("") + " " + sig['City'].fillna("") + " " + sig['State'].fillna("") + " " + sig['Zip Code'].fillna("")
sig['DBA'] = ''

In [10]:
df = fuzzy_join(sig.iloc[:100], reg, ['NAME','DBA'], 'ADDRESS', how = 'inner')
df

,Sponsor,Trade,NAME_l,Signatory Address,City_l,State_l,Zip Code_l,Date Signatory Added,Source,ADDRESS_l,...,Business is associated with an apprenticeship program,Business is sponsor of a program,Business is signatory to a group program,Business has Workers Compensation Insurance,Business is exempt from Workers Compensation Insurance,Georeference,ADDRESS_r,norm_NAME,norm_DBA,norm_ADDRESS
value,,,,,,,,,,,,,,,,,,,,,
429.0,Associated Builders and Contractors Empire State,Carpenter,"Ambuild Supply, LLC.",6605 Pittsford Palmyra Road,Fairport,NY,14450,3/17/2021,Group Signatory FOIL NYSDOL Oct 2024,6605 Pittsford Palmyra Road Fairport NY 14450,...,Yes,Yes,Yes,Yes,No,POINT (-77.44677 43.0693),6605 Pittsford Palmyra Rd #E10 Fairport NY 14450,ambuild supply llc,,6605 pittsford palmyra rd e10 fairport ny 14450
429.0,Associated Builders and Contractors Empire State,Carpenter,"Ambuild Supply, LLC.",6605 Pittsford Palmyra Road,Fairport,NY,14450,1/1/2024,Group Signatory FOIL NYSDOL Oct 2024,6605 Pittsford Palmyra Road Fairport NY 14450,...,Yes,Yes,Yes,Yes,No,POINT (-77.44677 43.0693),6605 Pittsford Palmyra Rd #E10 Fairport NY 14450,ambuild supply llc,,6605 pittsford palmyra rd e10 fairport ny 14450
1443.0,Associated Builders and Contractors Empire State,Carpenter,CMI Installations LLC.,718 County Highway 126,Amsterdam,NY,12010,3/10/2024,Group Signatory FOIL NYSDOL Oct 2024,718 County Highway 126 Amsterdam NY 12010,...,No,No,No,Yes,No,POINT (-74.15546 43.01931),718 Co Hwy 126 Amsterdam NY 12010,cmi installations llc,,718 co hwy 126 amsterdam ny 12010
2560.0,Associated Builders and Contractors Empire State,Carpenter,Gallo Construction Corp,50 Lincoln Ave,Watervliet,NY,12189,1/20/2022,Group Signatory FOIL NYSDOL Oct 2024,50 Lincoln Ave Watervliet NY 12189,...,Yes,Yes,Yes,Yes,No,POINT (-73.7172 42.71811),50 Lincoln Ave WATERVLIET NY 12189,gallo construction corporation,gallo construction corp,50 lincoln ave watervliet ny 12189
3928.0,Associated Builders and Contractors Empire State,Carpenter,"LoDuca Associates, Inc.",113 Division Ave,Blue Point,NY,11715,1/20/2022,Group Signatory FOIL NYSDOL Oct 2024,113 Division Ave Blue Point NY 11715,...,Yes,Yes,No,Yes,No,POINT (-73.03632 40.76085),113 Division Ave Blue Point NY 11715,loduca associates inc,loduca associates inc,113 division ave blue point ny 11715
3964.0,Associated Builders and Contractors Empire State,Carpenter,"Luck Bros., Inc.",73 Trade Road,Plattsburgh,NY,12901,1/20/2022,Group Signatory FOIL NYSDOL Oct 2024,73 Trade Road Plattsburgh NY 12901,...,Yes,No,Yes,Yes,No,POINT (-73.46484 44.71771),73 Trade Road Plattsburgh NY 12901,luck bros inc,,73 trade road plattsburgh ny 12901
4301.0,Associated Builders and Contractors Empire State,Carpenter,Milcon Construction Corp,142 Dale Street,West Babylon,NY,11704,4/25/2024,Group Signatory FOIL NYSDOL Oct 2024,142 Dale Street West Babylon NY 11704,...,Yes,No,No,Yes,No,POINT (-73.39146 40.73354),142 Dale Street West Babylon NY 11704,milcon construction corp,,142 dale street west babylon ny 11704
4603.0,Associated Builders and Contractors Empire State,Carpenter,Nickerson NY LLC,11 Moffitt Blvd,Bay Shore,NY,11706,1/22/2023,Group Signatory FOIL NYSDOL Oct 2024,11 Moffitt Blvd Bay Shore NY 11706,...,Yes,No,No,Yes,No,POINT (-73.23486 40.73141),11 Moffitt Boulevard Bay Shore NY 11706,nickerson ny llc,,11 moffitt boulevard bay shore ny 11706
4677.0,Associated Builders and Contractors Empire State,Carpenter,Northern Tier Contracting,329A Scotch Settlement Road,Gouverneur,NY,13642,3/17/2021,Group Signatory FOIL NYSDOL Oct 2024,329A Scotch Settlement Road Gouverneur NY 13642,...,No,No,No,Yes,No,POINT (-75.42571 44.35031),329A Scotch Settlement Rd. Gouverneur NY 13642,northern tier contracting inc,,329a scotch settlement rd gouverneur ny 13642


In [11]:
df[['NAME_l','ADDRESS_l','NAME_r','ADDRESS_r']]

,NAME_l,ADDRESS_l,NAME_r,ADDRESS_r
value,,,,
429.0,"Ambuild Supply, LLC.",6605 Pittsford Palmyra Road Fairport NY 14450,AmBuild Supply LLC,6605 Pittsford Palmyra Rd #E10 Fairport NY 14450
429.0,"Ambuild Supply, LLC.",6605 Pittsford Palmyra Road Fairport NY 14450,AmBuild Supply LLC,6605 Pittsford Palmyra Rd #E10 Fairport NY 14450
1443.0,CMI Installations LLC.,718 County Highway 126 Amsterdam NY 12010,CMI Installations LLC,718 Co Hwy 126 Amsterdam NY 12010
2560.0,Gallo Construction Corp,50 Lincoln Ave Watervliet NY 12189,Gallo Construction Corporation,50 Lincoln Ave WATERVLIET NY 12189
3928.0,"LoDuca Associates, Inc.",113 Division Ave Blue Point NY 11715,"LoDuca Associates, Inc.",113 Division Ave Blue Point NY 11715
3964.0,"Luck Bros., Inc.",73 Trade Road Plattsburgh NY 12901,Luck Bros. Inc,73 Trade Road Plattsburgh NY 12901
4301.0,Milcon Construction Corp,142 Dale Street West Babylon NY 11704,Milcon Construction Corp.,142 Dale Street West Babylon NY 11704
4603.0,Nickerson NY LLC,11 Moffitt Blvd Bay Shore NY 11706,"Nickerson NY, LLC",11 Moffitt Boulevard Bay Shore NY 11706
4677.0,Northern Tier Contracting,329A Scotch Settlement Road Gouverneur NY 13642,Northern Tier Contracting Inc,329A Scotch Settlement Rd. Gouverneur NY 13642
